In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, UpSampling2D
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

```python
TensorFlow is an open source machine learning framework developed by Google that is widely used in building and training deep neural networks.

MNIST is a dataset of handwritten digits that is commonly used for image classification tasks. It contains 60,000 training images and 10,000 test images.

Sequential and Model are two different ways to define a neural network architecture in Keras. Sequential is used for building a linear stack of layers, while Model allows for more complex network architectures with multiple inputs and outputs.

These are various types of layers that can be used to build a neural network architecture in Keras.

Conv2D is a 2D convolution layer that performs a convolution operation on the input data.

MaxPooling2D is a 2D pooling layer that performs a max pooling operation on the input data.

Flatten is a layer that flattens the output of the previous layer into a 1D vector.

Dense is a fully connected layer that connects every input neuron to every output neuron.

Dropout is a regularization technique that randomly drops out some of the neurons during training to prevent overfitting.

Input is a layer that specifies the shape of the input data.

UpSampling2D is a 2D upsampling layer that performs a nearest-neighbor interpolation on the input data to increase its size.

to_categorical is a function in Keras that converts a class vector (integers) to a binary class matrix.
 
NumPy is a Python library for numerical computing that is commonly used for data manipulation and scientific computing.

Matplotlib is a Python library for creating visualizations such as graphs, charts, and plots. It is commonly used for data visualization in machine learning and data science.


```

In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Rescale input data to range [0,1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


```python
This code loads the MNIST dataset from the Keras library. The dataset is split into two parts: a training set (x_train, y_train) and a test set (x_test, y_test).
The input data from the range of [0,255] to the range of [0,1]. This is done to normalize the data and make it easier for the neural network to learn from. The astype() method is used to convert the data type of the input data from int to float32.
```

In [ ]:
# Reshape input data to 4D tensor with shape (batch_size, height, width, channels)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Print dataset information
print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

```python
This code reshapes the input data from a 3D tensor with shape (batch_size, height, width) to a 4D tensor with shape (batch_size, height, width, channels). In this case, the channels dimension is set to 1 because the input images are grayscale. If the images were RGB, the channels dimension would be set to 3.

```

In [ ]:
# Build autoencoder model
input_img = Input(shape=(28,28,1))
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)

```python
This code builds an autoencoder model using the Keras library. The model has an input layer of shape (28, 28, 1), which corresponds to the shape of the input images. The model consists of three convolutional layers with 32, 64, and 128 filters respectively. Each convolutional layer is followed by a max pooling layer. The output of the final max pooling layer is the encoded representation of the input images. The encoded representation is then decoded by three convolutional layers with 128, 64, and 32 filters respectively. Each convolutional layer is followed by an upsampling layer. The output of the final upsampling layer is the decoded images. The activation function used for the convolutional layers is ReLU, and the activation function used for the output layer is sigmoid.

Note that this autoencoder model is designed to compress the input images into a lower dimensional representation and then reconstruct them from the compressed representation.
```

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


```python
This code creates an instance of the autoencoder model by specifying the input and output layers using the Model class from Keras. The input layer is defined as input_img, which is of shape (28, 28, 1). The output layer is defined as decoded, which is the result of the last convolutional layer.
The compile method is used to configure the model for training. In this case, the optimizer used is adam, which is an algorithm for stochastic optimization. The loss function used is binary_crossentropy, which is a popular choice for binary classification problems, where the goal is to minimize the difference between the predicted output and the true output.
```

In [ ]:
# Train autoencoder model
autoencoder.fit(x_train, x_train, epochs=10, batch_size=32, validation_data=(x_test, x_test))


```python
This code trains the autoencoder model using the fit method from Keras. The input and output data are both x_train, which are the preprocessed images of the MNIST dataset. The epochs parameter specifies the number of times to iterate over the entire training dataset. The batch_size parameter specifies the number of samples to use in each batch of training.

The validation_data parameter specifies the data on which to evaluate the loss and any model metrics at the end of each epoch. In this case, the validation data is set to x_test, which is the preprocessed test set of the MNIST dataset.

During training, the model learns to minimize the binary cross-entropy loss between the predicted output and the true output. The model parameters are updated using the adam optimizer specified during the compilation step. After training is complete, the trained autoencoder model can be used to reconstruct images or for other downstream tasks such as feature extraction.


```

In [ ]:
# Extract features from input data using the trained autoencoder
encoder = Model(input_img, encoded)
x_train_features = encoder.predict(x_train)
x_test_features = encoder.predict(x_test)

# Convert class labels to one-hot encoded vectors
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

```python
This code extracts features from the input data using the trained autoencoder model. The encoder model is created by specifying the input layer as input_img and the output layer as encoded. This model maps the input images to the encoded features learned by the autoencoder.

The predict method is then used to generate encoded features for the training and test datasets. These encoded features can be used for downstream tasks such as classification or clustering.

Finally, the class labels y_train and y_test are converted to one-hot encoded vectors using the to_categorical method from Keras. This is often required for classification tasks where the class labels are represented as integers but need to be converted to a vector representation for training machine learning models.




```

In [ ]:
# Build CNN model
model = Sequential()
model.add(Flatten(input_shape=(4,4,128))) # Input shape is (batch_size, height, width, channels) after feature extraction
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10, activation='softmax'))


```python
This code builds a CNN model for classification. The input to the model is the encoded features extracted using the autoencoder, which have a shape of (batch_size, height, width, channels) after feature extraction.

The Flatten layer is used to flatten the encoded features into a 1D vector so that they can be fed into the fully connected layers. The Dense layer with 128 units and ReLU activation function is added after the Flatten layer, followed by a Dropout layer with a dropout rate of 0.5 to prevent overfitting.

Finally, the output layer is a Dense layer with 10 units (since there are 10 classes in the MNIST dataset) and a softmax activation function, which is used to produce the class probabilities for each input image.
```

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


```python
This code compiles the CNN model with the Adam optimizer and categorical cross-entropy loss function, which is commonly used for multi-class classification problems. The accuracy metric is also specified to track the performance of the model during training and evaluation.
```

In [ ]:
# Train the model
history = model.fit(x_train_features, y_train, epochs=10, batch_size=32, validation_data=(x_test_features, y_test))


```python
This code trains the compiled CNN model using the extracted features from the trained autoencoder (x_train_features and x_test_features) as input data and the one-hot encoded class labels (y_train and y_test) as target data. The model is trained for 10 epochs with a batch size of 32 and the validation data is used to evaluate the model's performance after each epoch. The training history is stored in the history object.
```

In [ ]:
# Print training and testing accuracy
train_acc = history

```python
The variable train_acc is not assigned any value in the code you provided. To print the training and testing accuracy, you can use the history object that was created during training
This code uses the history object to extract the final training and testing accuracy from the training process. The history.history attribute is a dictionary that contains the loss and metrics values recorded during training. The 'accuracy' key corresponds to the training accuracy and the 'val_accuracy' key corresponds to the testing accuracy. The [-1] index is used to extract the last value recorded for each metric. The f-string is used to print the accuracy values with 4 decimal places.
```

In [ ]:
# Select a random image from test set and predict its label
import random

index = random.randint(0, x_test.shape[0]-1)
img = x_test[index]
label = y_test[index]
print('True label:', np.argmax(label))

# Reshape input data to 4D tensor with shape (1, height, width, channels)
img = img.reshape(1, 28, 28, 1)

# Extract features from input image using the trained autoencoder
img_features = encoder.predict(img)

# Predict label using the trained CNN model
prediction = model.predict(img_features)[0]
print('Predicted label:', np.argmax(prediction))

# Display the input image
plt.imshow(x_test[index], cmap='gray')
plt.show()
print('Input image:')
plt.show()

# Display the reconstructed image using the trained autoencoder
reconstructed_img = autoencoder.predict(img)
plt.imshow(reconstructed_img[0], cmap='gray')
plt.show()
print('Reconstructed image using autoencoder:')